<div align='center'>
    <p>You can open this file in <b>Google Colab</b></p>
    <a href="https://colab.research.google.com/github/vstark21/Neural_Style_Transfer/blob/master/main.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg"></a>
</div>

First lets import our modules.

In [16]:
from main_utils import *
import time
import IPython.display as display

Now lets get our data.

In [17]:
content_image, style_image = get_data("photo.jpg", "style.jpg")

content_image_copy = content_image.copy()

Create a class and function to get loss and gradients to optimize.

In [18]:
# Evaluator class for optimizing 
class Evaluator:

    def loss(self, x):

        loss_value, grad_values = loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values

        return self.loss_value

    def grads(self, x):

        grad_values = np.copy(self.grad_values)
        return grad_values

# This returns loss and gradients 
def loss_and_grads(generated):

    generated = generated.reshape((1, 512, 512, 3))
    outs = f_outputs([generated])
    loss_value = outs[0]
    grad_values = np.array(outs[1]).flatten().astype('float64')
    return loss_value, grad_values

Now lets create  variables and constant in the computational graph.

In [19]:
# Disabling eager exceution to use K.Gradients function
tf.compat.v1.disable_eager_execution()

# Creating variables and constant in the computational flow graph
content_image = K.constant(content_image)
style_image = K.constant(style_image)
generated_image = K.placeholder((1, 512, 512, 3))
loss = K.variable(0.)

# Get the layers needed to prepare the loss metric
content_layer, style_layers = get_layers(content_image, style_image, generated_image)

# Define loss and gradient
loss = total_loss(content_layer, style_layers, generated_image)

# Define a function to get loss and gradients
f_outputs = K.function([generated_image], [loss, K.gradients(loss, generated_image)])

And now we start optimization process.

In [ ]:
# Creating evaluator object
evaluator = Evaluator()

# Assigning content image to generated image
generated_img = content_image_copy

for i in range(20):
    print('In iteration no:', i)

    start_time = time.time()
    generated_img, min_val, info = fmin_l_bfgs_b(evaluator.loss, generated_img.flatten(),
                                                     fprime=evaluator.grads, maxfun=300)
    
    if i % 5 == 0:
        generated_image = generated_img.copy()
        generated_image = generated_image.reshape(512, 512, 3)
        generated_image = np.clip(generated_image, 0, 255).astype('uint8')
        cv2.imwrite("result-16 " + str(i) + ".jpg", generated_image)
    
    end_time = time.time()
    display.clear_output(wait=True)

    print(f'Loss: {round(min_val / 10**6, 3)} x 10^6')

    print(f'Iteration {i} took {end_time - start_time} seconds')